### Implementing FunkSVD - Solution

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 3 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 59.987408
2 		 47.921640
3 		 32.358851
4 		 17.333742
5 		 7.291920
6 		 2.679182
7 		 1.113507
8 		 0.666207
9 		 0.540428
10 		 0.499506


In [4]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  7.74776436   9.11966044   8.33898143]
 [  6.65063976   8.40742314   7.62311961]
 [  7.5468509    9.22298639   8.35381938]
 [  8.40380947  10.0863493    9.63394068]
 [  8.08473425   9.81941984   8.8024718 ]
 [  7.80768829   9.24728772   8.55355712]]
[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations. Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 3 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 300.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about the `mean squared error` at the end of each training iteration?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=300)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 53.070099
2 		 38.251685
3 		 22.068117
4 		 9.733060
5 		 3.453885
6 		 1.241427
7 		 0.642307
8 		 0.497952
9 		 0.460262
10 		 0.446528
11 		 0.438715
12 		 0.432676
13 		 0.427247
14 		 0.422005
15 		 0.416766
16 		 0.411445
17 		 0.406004
18 		 0.400424
19 		 0.394699
20 		 0.388827
21 		 0.382808
22 		 0.376643
23 		 0.370337
24 		 0.363891
25 		 0.357312
26 		 0.350603
27 		 0.343770
28 		 0.336820
29 		 0.329759
30 		 0.322597
31 		 0.315341
32 		 0.308002
33 		 0.300588
34 		 0.293112
35 		 0.285585
36 		 0.278019
37 		 0.270426
38 		 0.262819
39 		 0.255213
40 		 0.247621
41 		 0.240055
42 		 0.232532
43 		 0.225063
44 		 0.217664
45 		 0.210346
46 		 0.203124
47 		 0.196010
48 		 0.189016
49 		 0.182153
50 		 0.175432
51 		 0.168862
52 		 0.162452
53 		 0.156210
54 		 0.150144
55 		 0.144258
56 		 0.138558
57 		 0.133047
58 		 0.127729
59 		 0.122605
60 		 0.117676
61 		 0.112941
62 		 0.108401
63 		 0.104052
64 	

In [6]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  7.01751095  10.00682781   7.98117957]
 [  5.99264412   9.9970382    7.00771669]
 [  7.98270058   8.99286703   8.01879472]
 [  8.00157841  10.00091433   9.99818448]
 [  9.00385028   9.00174064   8.99570131]
 [  8.00097043   9.00019662   8.9991639 ]]
[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


**In this case, we were able to completely reconstruct the item-movie matrix to closer to 0 mean squared error at the end of 300th iterations.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 3 latent features, a learning rate of 0.005, and 450 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,   8.],
        [  6.,  10.,   7.],
        [  8.,   9.,   8.],
        [  8.,  10.,  10.],
        [  9.,   9.,   9.],
        [  8.,   9.,   9.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=450)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 58.356599
2 		 45.166780
3 		 29.457275
4 		 15.507451
5 		 6.752270
6 		 2.766053
7 		 1.299750
8 		 0.791162
9 		 0.600118
10 		 0.517157
11 		 0.475776
12 		 0.452710
13 		 0.438553
14 		 0.429000
15 		 0.421905
16 		 0.416142
17 		 0.411102
18 		 0.406447
19 		 0.401986
20 		 0.397611
21 		 0.393263
22 		 0.388907
23 		 0.384523
24 		 0.380102
25 		 0.375639
26 		 0.371134
27 		 0.366588
28 		 0.362002
29 		 0.357379
30 		 0.352724
31 		 0.348039
32 		 0.343328
33 		 0.338596
34 		 0.333848
35 		 0.329087
36 		 0.324318
37 		 0.319545
38 		 0.314774
39 		 0.310008
40 		 0.305253
41 		 0.300513
42 		 0.295792
43 		 0.291095
44 		 0.286427
45 		 0.281790
46 		 0.277190
47 		 0.272630
48 		 0.268115
49 		 0.263646
50 		 0.259229
51 		 0.254866
52 		 0.250560
53 		 0.246313
54 		 0.242129
55 		 0.238009
56 		 0.233956
57 		 0.229970
58 		 0.226055
59 		 0.222210
60 		 0.218438
61 		 0.214738
62 		 0.211112
63 		 0.207560
64 

In [9]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is: {}".format(preds[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 7. Not bad!")

The predicted value for the missing rating is: 6.880493593174232

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 7. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 3 latent features, 0.005 learning rate, and 500 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=500)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 22.814941
2 		 10.638300
3 		 7.358131
4 		 5.708696
5 		 4.696700
6 		 4.005666
7 		 3.500587
8 		 3.113243
9 		 2.805251
10 		 2.553430
11 		 2.343050
12 		 2.164384
13 		 2.010821
14 		 1.877722
15 		 1.761709
16 		 1.660206
17 		 1.571161
18 		 1.492880
19 		 1.423931
20 		 1.363085
21 		 1.309277
22 		 1.261577
23 		 1.219174
24 		 1.181359
25 		 1.147517
26 		 1.117115
27 		 1.089696
28 		 1.064867
29 		 1.042293
30 		 1.021688
31 		 1.002808
32 		 0.985446
33 		 0.969423
34 		 0.954586
35 		 0.940804
36 		 0.927961
37 		 0.915959
38 		 0.904712
39 		 0.894145
40 		 0.884192
41 		 0.874797
42 		 0.865910
43 		 0.857487
44 		 0.849490
45 		 0.841886
46 		 0.834645
47 		 0.827741
48 		 0.821153
49 		 0.814859
50 		 0.808841
51 		 0.803083
52 		 0.797570
53 		 0.792288
54 		 0.787224
55 		 0.782366
56 		 0.777704
57 		 0.773227
58 		 0.768924
59 		 0.764787
60 		 0.760807
61 		 0.756975
62 		 0.753284
63 		 0.749725
64 		

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [11]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 14287.

The number of ratings made for user-movie pairs that didn't have ratings is 13835713


In [14]:
# Test your results against the solution
assert num_ratings == 14287, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 13835713, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
